In [1]:
import pdf
import util
import config
import argparse
from tqdm import tqdm
import vector_store as vs
from openai import OpenAI
from langchain_qdrant import FastEmbedSparse
from langchain_openai import OpenAIEmbeddings
import api
import retrieval
import chatbot
import policy
import uuid
import json
from collections import defaultdict
import agent
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap
import hdbscan
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from qdrant_client.http.models import Filter, FieldCondition, Range, MatchValue, MatchAny
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import pandas as pd
import os

# Extract content

In [3]:
source_folders = ['Dataset/COP Meetings', 'Dataset/G7/Canada', 'Dataset/G7/France', 'Dataset/G7/Germany', 'Dataset/G7/Italy', 'Dataset/G7/Japan', 'Dataset/G7/United Kingdom', 'Dataset/G7/United States', 'Dataset/IPCC report/6th assessment']
pdf_paths = []
for source_folder in source_folders:
    pdf_paths.extend(util.get_individual_file_path(source_folder, '.pdf'))

util.clear_output_folder(config.output_folder)
for file_path in pdf_paths:
    file = pdf.PDF(file_path)
    file.extract_text(config.output_folder)

Extracted_contents is cleared!
Extracted_contents is ready for new content.
Extracting texts from TWN - COP29.
Extracted text saved to: Extracted_contents/COP Meetings/TWN - COP29.txt
Extracting texts from ECO-15-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-15-11-2024.txt
Extracting texts from ECO-22-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-22-11-2024.txt
Extracting texts from ECO-19-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-19-11-2024.txt
Extracting texts from ECO-21-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-21-11-2024.txt
Extracting texts from ECO-13-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-13-11-2024.txt
Extracting texts from ECO-16-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-16-11-2024.txt
Extracting texts from ECO-18-11-2024.
Extracted text saved to: Extracted_contents/COP Meetings/ECO-18-11-2024.txt
Extracting texts f

# Chunking Records

In [4]:
content_folders = ['Extracted_contents/COP Meetings', 'Extracted_contents/G7/Canada', 'Extracted_contents/G7/France', 'Extracted_contents/G7/Germany', 'Extracted_contents/G7/Italy', 'Extracted_contents/G7/Japan', 'Extracted_contents/G7/United Kingdom', 'Extracted_contents/G7/United States']
txt_paths = []
for content_folder in content_folders:
    txt_paths.extend(util.get_individual_file_path(content_folder, '.txt'))

for file_path in txt_paths:
    file = pdf.PDF(file_path)
    file.load_content(file_path)
    file.naive_chunking(1000, 100)
    for chunk in tqdm(file.chunks, desc=f"Judging {file.pdf_name}'s chunks revelance and summarizing"):
        chunk.classify_relevance()
        if chunk.relevance:
            chunk.summarize_record()
    file.filter_chunks_by_revelance()
    file.save_chunks(config.chunks_folder)

ECO-21-11-2024's content is loaded.
Total text chunks created: 6


Judging ECO-21-11-2024's chunks revelance and summarizing: 100%|██████████| 6/6 [00:12<00:00,  2.14s/it]
Filtering ECO-21-11-2024's chunks.: 100%|██████████| 6/6 [00:00<00:00, 146312.93it/s]


0 chunks are filtered out!
Data written to Chunks/COP Meetings/ECO-21-11-2024.json
ECO-13-11-2024's content is loaded.
Total text chunks created: 3


Judging ECO-13-11-2024's chunks revelance and summarizing: 100%|██████████| 3/3 [00:10<00:00,  3.55s/it]
Filtering ECO-13-11-2024's chunks.: 100%|██████████| 3/3 [00:00<00:00, 65196.44it/s]


0 chunks are filtered out!
Data written to Chunks/COP Meetings/ECO-13-11-2024.json
ECO-16-11-2024's content is loaded.
Total text chunks created: 7


...

0 chunks are filtered out!
Data written to Chunks/G7/United States/Climate Adaptation Plan.json
Project 2025's content is loaded.
Total text chunks created: 599


Judging Project 2025's chunks revelance and summarizing: 100%|██████████| 599/599 [30:31<00:00,  3.06s/it]
Filtering Project 2025's chunks.: 100%|██████████| 599/599 [00:00<00:00, 1483109.86it/s]


0 chunks are filtered out!
Data written to Chunks/G7/United States/Project 2025.json
Trump Environmental Accomplishments first term's content is loaded.
Total text chunks created: 14


Judging Trump Environmental Accomplishments first term's chunks revelance and summarizing: 100%|██████████| 14/14 [01:24<00:00,  6.07s/it]
Filtering Trump Environmental Accomplishments first term's chunks.: 100%|██████████| 14/14 [00:00<00:00, 53188.64it/s]

0 chunks are filtered out!
Data written to Chunks/G7/United States/Trump Environmental Accomplishments first term.json


# Chunking Knowledges

In [8]:
content_folders = ['Extracted_contents/IPCC report/6th assessment']
txt_paths = []
for content_folder in content_folders:
    txt_paths.extend(util.get_individual_file_path(content_folder, '.txt'))

for file_path in txt_paths:
    file = pdf.PDF(file_path)
    file.load_content(file_path)
    file.naive_chunking(5000, 500)
    for chunk in tqdm(file.chunks, desc=f"Summarizing knowledges of {file.pdf_name}"):
        chunk.summarize_knowledge()
    file.save_chunks(config.chunks_folder)

Synthesis report's content is loaded.
Total text chunks created: 39


Summarizing knowledges of Synthesis report: 100%|██████████| 39/39 [04:26<00:00,  6.82s/it]


Data written to Chunks/IPCC report/6th assessment/Synthesis report.json
WG2 - Impacts, Adaptation and Vulnerability's content is loaded.
Total text chunks created: 1167


Summarizing knowledges of WG2 - Impacts, Adaptation and Vulnerability: 100%|██████████| 1167/1167 [1:47:53<00:00,  5.55s/it] 


Data written to Chunks/IPCC report/6th assessment/WG2 - Impacts, Adaptation and Vulnerability.json
WG3 - Mitigation of Climate Change's content is loaded.
Total text chunks created: 757


Summarizing knowledges of WG3 - Mitigation of Climate Change: 100%|██████████| 757/757 [1:11:41<00:00,  5.68s/it]


Data written to Chunks/IPCC report/6th assessment/WG3 - Mitigation of Climate Change.json
WG1- Physical Science Basis's content is loaded.
Total text chunks created: 902


Summarizing knowledges of WG1- Physical Science Basis: 100%|██████████| 902/902 [1:17:49<00:00,  5.18s/it]

Data written to Chunks/IPCC report/6th assessment/WG1- Physical Science Basis.json


# Extract Policies

In [2]:
util.clear_output_folder('Extracted_policies')
policy_folders = ['Chunks/G7/Canada', 'Chunks/G7/France', 'Chunks/G7/Germany', 'Chunks/G7/Italy', 'Chunks/G7/Japan', 'Chunks/G7/United Kingdom', 'Chunks/G7/United States']
json_paths = []
for folder in policy_folders:
    json_paths.extend(util.get_individual_file_path(folder, '.json'))


for file in json_paths:
    records = util.load_json(file)
    for item in records:
        pl = policy.Policy(f"{uuid.uuid4()}")
        pl.load_policy(item)
        pl.save_policy('Extracted_policies/Policies.json')

Extracted_policies is cleared!
Extracted_policies is ready for new content.
Loaded content from Chunks/G7/Canada/Federal actions for a clean growth economy.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Data written to Extracted_policies/Policies.json
Loaded content from Chunks/G7/Canada/Pan-Canadian Frame

# Fill Nan values of Policies

## Embed original contents

In [ ]:
chunking_folders = ['Chunks/COP Meetings', 'Chunks/G7/Canada', 'Chunks/G7/France', 'Chunks/G7/Germany', 'Chunks/G7/Italy', 'Chunks/G7/Japan', 'Chunks/G7/United Kingdom', 'Chunks/G7/United States']
chunk_paths = []
for chunking_folder in chunking_folders:
    chunk_paths.extend(util.get_individual_file_path(chunking_folder, '.json'))

collection_name = 'Original_Contents'
vs.create_collection(collection_name, 1536)

for i, chunk_file in enumerate(chunk_paths):
    print(f'Embedding file {i}/{len(chunk_paths)}')
    file = pdf.PDF(chunk_file)
    file.chunks = util.create_class_from_json(pdf.Chunk, chunk_file)
    vs.add_chunk(file.chunks, collection_name)

# Regenerate Nan Values

In [3]:
dense_embedding_function = OpenAIEmbeddings(api_key=api.OPENAI_API, model="text-embedding-3-small")
sparse_embedding_function = FastEmbedSparse(model_name="prithivida/Splade_PP_en_v1")

collection = vs.get_collection(
    'Original_Contents', 
    dense_embedding_function,
    sparse_embedding_function
)

In [ ]:
policies = util.load_json('Extracted_policies/Policies.json')

for pl in tqdm(policies):
    if pl['country'] == 'Nan':
        query = f'I have a policy description as the following, please find out which country it comes from: {pl['policy'], pl['effect']}. Use the structure Country:, with exactly the country name'

        retriever = retrieval.Retriever(query, collection)
        retriever.similarity_search(10)
        retriever.rerank('content')
        retriever.cos_filtering('content', 0.8, 3)

        context = " "
        for filtered_context in retriever.filtered_contents:
            context += filtered_context

        response = chatbot.respond(query, context)
        pl['country'] = response.split('Country:')[-1]

    if pl['year'] == 'Nan':
        query = f'I have a policy description as the following, please find out which date it comes from: {pl['policy'], pl['effect']}. Use the structure Year:, with exactly the year number'

        retriever = retrieval.Retriever(query, collection)
        retriever.similarity_search(10)
        retriever.rerank('content')
        retriever.cos_filtering('content', 0.8, 3)

        context = " "
        for filtered_context in retriever.filtered_contents:
            context += filtered_context

        response = chatbot.respond(query, context)
        pl['year'] = response.split('Year:')[-1].strip()

with open('Extracted_policies/Policies.json', 'w') as f:
    json.dump(policies, f, indent=4)

Loaded content from Extracted_policies/Policies.json


100%|██████████| 1683/1683 [38:35<00:00,  1.38s/it] 


# Combine all the policies

In [6]:
file1 = "Extracted_policies/Policies.json"
file2 = "Dataset/Climate_Policies/climate_policies.json"

with open(file1, "r", encoding="utf-8") as f1:
    data1 = json.load(f1)
with open(file2, "r", encoding="utf-8") as f2:
    data2 = json.load(f2)
combined_data = data1 + data2

with open("Extracted_policies/Combined_Policies.json", "w", encoding="utf-8") as out_file:
    json.dump(combined_data, out_file, indent=4, ensure_ascii=False)

# Group Policies

In [7]:
util.clear_output_folder('Policies_by_Country')
util.clear_output_folder('Policies_by_Year')
policy.group_by('country', 'Extracted_policies/Combined_Policies.json', 'Policies_by_Country')
policy.group_by('year', 'Extracted_policies/Combined_Policies.json', 'Policies_by_Year')

Policies_by_Country is cleared!
Policies_by_Country is ready for new content.
Policies_by_Year is cleared!
Policies_by_Year is ready for new content.
Grouped data has been saved in 'Policies_by_Country' directory.
Grouped data has been saved in 'Policies_by_Year' directory.


# Embed Policies and Knowledges

In [8]:
policy_collection_name = 'Policies'
vs.create_collection(policy_collection_name, 1536)
policies = util.load_json('Extracted_policies/Policies.json')
vs.add_policies(policies, policy_collection_name)

Deleted old version collection Policies
Collection Policies initialized.
Loaded content from Extracted_policies/Policies.json


Embedding policies and storing the embeddings.: 100%|██████████| 1683/1683 [35:16<00:00,  1.26s/it] 

All chunks are saved to Policies


In [ ]:
knowledge_collection_name = 'Knowldeges'
vs.create_collection(knowledge_collection_name, 1536)

knowledge_folders = ['Chunks/IPCC report/6th assessment']
knowledge_paths = []
for folder in knowledge_folders:
    knowledge_paths.extend(util.get_individual_file_path(folder, '.json'))

for knowledge_file in knowledge_paths:
    knowledges = util.load_json(knowledge_file)
    vs.add_knowledges(knowledges, knowledge_collection_name)

Deleted old version collection Knowldeges
Collection Knowldeges initialized.
Loaded content from Chunks/IPCC report/6th assessment/WG2 - Impacts, Adaptation and Vulnerability.json


Embedding knowledges and storing the embeddings.: 100%|██████████| 1167/1167 [26:34<00:00,  1.37s/it]


All chunks are saved to Knowldeges
Loaded content from Chunks/IPCC report/6th assessment/WG3 - Mitigation of Climate Change.json


Embedding knowledges and storing the embeddings.: 100%|██████████| 757/757 [14:34<00:00,  1.16s/it]


All chunks are saved to Knowldeges
Loaded content from Chunks/IPCC report/6th assessment/Synthesis report.json


...

All chunks are saved to Knowldeges
Loaded content from Chunks/IPCC report/6th assessment/WG1- Physical Science Basis.json


Embedding knowledges and storing the embeddings.: 100%|██████████| 902/902 [17:41<00:00,  1.18s/it]


All chunks are saved to Knowldeges
Loaded content from Chunks/IPCC report/6th assessment/Summary for policymakers.json


Embedding knowledges and storing the embeddings.: 100%|██████████| 81/81 [01:35<00:00,  1.18s/it]

All chunks are saved to Knowldeges


# Respond to a Query

In [8]:
policy_collection_name = 'Policies'
knowledge_collection_name = 'Knowldeges'

def respond_text_query(
        query, 
        txt_similarity_topk, 
        knowledge_similarity_topk,
        cos_filtering_threshold, 
        txt_cos_filtering_topk,
        knowledge_cos_filtering_topk
        ):
    
    dense_embedding_function = OpenAIEmbeddings(api_key=api.OPENAI_API, model="text-embedding-3-small")
    sparse_embedding_function = FastEmbedSparse(model_name="prithivida/Splade_PP_en_v1")
    
    collection_policy = vs.get_collection(
        policy_collection_name, 
        dense_embedding_function,
        sparse_embedding_function
    )

    collection_knowledge = vs.get_collection(
        knowledge_collection_name, 
        dense_embedding_function,
        sparse_embedding_function
    )

    policy_retriever = retrieval.Retriever(query, collection_policy)
    policy_retriever.similarity_search(txt_similarity_topk)
    policy_retriever.rerank(['policy','effect'])
    policy_retriever.cos_filtering(['policy','effect'], cos_filtering_threshold, txt_cos_filtering_topk)

    context = " "
    for filtered_context in policy_retriever.filtered_contents:    
        context += filtered_context

    knowledge_retriever = retrieval.Retriever(query, collection_knowledge)
    knowledge_retriever.similarity_search(knowledge_similarity_topk)
    knowledge_retriever.rerank('content')
    knowledge_retriever.cos_filtering('content', cos_filtering_threshold, knowledge_cos_filtering_topk)

    knowledge = " "
    for filtered_context in knowledge_retriever.filtered_contents:
        knowledge += filtered_context
    
    response = chatbot.answer_with_knowledge(query, context, knowledge)

    return response

In [9]:
response = respond_text_query(
        query = 'Can you predict the future policy of 2050?', 
        txt_similarity_topk = 30,
        knowledge_similarity_topk = 20,
        cos_filtering_threshold = 0.8,
        txt_cos_filtering_topk = 20,
        knowledge_cos_filtering_topk = 15
        )
print(response)

Predicting the specific climate policies of 2050 is challenging, but based on the provided context, we can infer several likely trends. 

The Climate Action Plan 2050 outlines ambitious targets to reduce greenhouse gas emissions by 80 to 95 percent compared to 1990 levels. This indicates a strong commitment to achieving greenhouse gas neutrality by 2050, emphasizing the role of renewable energy sources and energy efficiency. There is also a focus on restructuring industries, particularly in the automotive sector, to align with these goals.

Moreover, policies such as the Energy Efficiency Strategy 2050 suggest a roadmap that aims to halve primary energy consumption, indicating a clear trajectory towards reduced energy use overall. Initiatives to promote electric mobility and shift freight transportation modes also illustrate a shift in logistical and commuting behaviors expected by 2050. 

In summary, while precise predictions are not possible, the context suggests that by 2050, polici

# Conference Simulation

In [8]:
dense_embedding_function = OpenAIEmbeddings(api_key=api.OPENAI_API, model="text-embedding-3-small")
sparse_embedding_function = FastEmbedSparse(model_name="prithivida/Splade_PP_en_v1")

policy_collection = vs.get_collection(
    'Policies', 
    dense_embedding_function,
    sparse_embedding_function
)

knowledge_collection = vs.get_collection(
    'Knowldeges', 
    dense_embedding_function,
    sparse_embedding_function
)
stances = {
    "India": "Focused on rapid renewable energy expansion with a strong emphasis on solar power and sustainable development in a growing economy.",
    "France": "Leveraging its robust nuclear energy legacy while increasingly investing in renewables to drive a low-carbon transition.",
    "Japan": "Balancing energy security with cautious progress in renewables, while reassessing nuclear policy post-Fukushima.",
    "Canada": "Navigating a transition from a fossil-fuel–rich past toward ambitious renewable targets and enhanced environmental stewardship.",
    "Brazil": "Harnessing abundant natural resources for biofuels and sustainable agriculture while addressing deforestation and ecosystem protection.",
    "Norway": "A leader in renewable energy—especially hydropower and electric mobility—while managing its substantial oil and gas interests.",
    "United Kingdom": "Pioneering in decarbonization through offshore wind and comprehensive net zero strategies, driving a sustainable energy future."
}
countries_list = ["India", "France", "Japan", "Canada", "Brazil", "Norway", "United Kingdom"]
shared_goal = "Reduce carbon emission by 30/% by 2050."
result = agent.multi_agent_climate_discussion(countries_list, shared_goal, policy_collection, knowledge_collection, 2000, stances)
print(result)

{'India': '### Proposed Climate Policy for India: "Green Horizon 2050"\n\n#### Overview\nThe "Green Horizon 2050" policy aims to position India as a leader in renewable energy, with a specific focus on solar power, while ensuring sustainable development and social equity. The policy is designed to meet the shared goal of reducing carbon emissions by 30% by 2050 and to promote economic growth, job creation, and energy security.\n\n#### Key Components\n\n1. **Solar Power Expansion**\n   - **Targeted Solar Capacity**: Set a target of 500 GW of solar power capacity by 2030, with a phased approach to achieve 1,000 GW by 2050.\n   - **Incentives for Solar Installations**: Provide financial incentives for residential, commercial, and industrial solar installations, including subsidies, tax breaks, and low-interest loans.\n   - **Solar Parks and Rooftop Initiatives**: Expand the development of solar parks and promote rooftop solar installations through streamlined permitting processes and net 

In [10]:
def unescape_string(text):
    return text.encode('utf-8').decode('unicode_escape')

for key, value in result.items():
    result[key] = unescape_string(value)

print("=== India Policy ===")
print(result["India"])
print("\n------------------------------\n")
print("=== France Policy ===")
print(result["France"])
print("=== Japan Policy ===")
print(result["Japan"])
print("\n------------------------------\n")
print("=== Canada Policy ===")
print(result["Canada"])
print("=== Brazil Policy ===")
print(result["Brazil"])
print("\n------------------------------\n")
print("=== Norway Policy ===")
print(result["Norway"])
print("\n------------------------------\n")
print("=== UK Policy ===")
print(result["United Kingdom"])

=== India Policy ===
### Proposed Climate Policy for India: "Green Horizon 2050"

#### Overview
The "Green Horizon 2050" policy aims to position India as a leader in renewable energy, with a specific focus on solar power, while ensuring sustainable development and social equity. The policy is designed to meet the shared goal of reducing carbon emissions by 30% by 2050 and to promote economic growth, job creation, and energy security.

#### Key Components

1. **Solar Power Expansion**
   - **Targeted Solar Capacity**: Set a target of 500 GW of solar power capacity by 2030, with a phased approach to achieve 1,000 GW by 2050.
   - **Incentives for Solar Installations**: Provide financial incentives for residential, commercial, and industrial solar installations, including subsidies, tax breaks, and low-interest loans.
   - **Solar Parks and Rooftop Initiatives**: Expand the development of solar parks and promote rooftop solar installations through streamlined permitting processes and net 

# Topic Modelling

## What is the common topic in some years?

In [105]:
policies = []
years = [item for item in range(2011, 2016)]

for year in years:
  policies.extend(util.load_json(f'Policies_by_Year/{year}.json'))

vectorizer_model = CountVectorizer(
    stop_words='english',   
    ngram_range=(1,2),
    min_df=2       
)

docs = [policy["policy"]+policy["effect"] for policy in policies]
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

umap_model = umap.UMAP(
    n_neighbors=2,
    n_components=2,
    min_dist=0.0,
    metric='cosine',
    random_state=42,
    init="random" 
)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2))
ctfidf_model = ClassTfidfTransformer()

topic_model = BERTopic(
  n_gram_range = (1,2),
  embedding_model=embedding_model,    
  hdbscan_model=hdbscan_model,        
  vectorizer_model=vectorizer_model,  
  umap_model=umap_model,
  ctfidf_model=ctfidf_model,          
  calculate_probabilities=True,        
  verbose=True
)

topics, probs = topic_model.fit_transform(docs)
topic_model_info = topic_model.get_topic_info()

2025-02-19 19:14:25,509 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

2025-02-19 19:14:31,581 - BERTopic - Embedding - Completed ✓
2025-02-19 19:14:31,582 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-19 19:14:32,463 - BERTopic - Dimensionality - Completed ✓
2025-02-19 19:14:32,464 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-19 19:14:32,509 - BERTopic - Cluster - Completed ✓
2025-02-19 19:14:32,515 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-19 19:14:32,650 - BERTopic - Representation - Completed ✓


In [102]:
fig = topic_model.visualize_topics()
fig.show()

In [103]:
bar_chart = topic_model.visualize_barchart()
bar_chart.show()

In [97]:
representative_docs = topic_model.get_representative_docs(3)
df = pd.DataFrame(representative_docs)
pd.set_option('display.max_colwidth', None)
df

,0
0,"Reorientation of USAID's focus from budget formulation to execution of appropriated resources, appointment of political appointees in key positions, and restructuring of the Bureau for Policy, Planning, and Learning.Streamlined operations within USAID, faster execution of foreign assistance, and alignment of agency policies with the new Administration's viewpoint."
1,Expansion of USAID's contracting and agreement officers and restoration of the Senior Official Accountability Review (SOAR) process; emphasis on localization and increased funding to local entities; expansion of the New Partnership Initiative (NPI); updating the Global Health Bureau’s portfolio.Accelerated delivery of funds for disaster responses; increased funding disbursed to local entities from about 25% to nearly 70% during the Trump Administration with positive results; improved coordination with local partners and a comprehensive approach to global health.
2,"Review and amend or withdraw major agency policies within the first calendar year of the new Administration; prioritize conservative appointments and messaging at USAID; issue clear guidance on eligibility of faith-based organizations for funding; rapid staffing of key positions in the Bureau for Democracy, Development, and Innovation (DDI); focus foreign aid strategy on countering Communist China's influence in Asia.The policies aim to reshape USAID's operations and priorities to align with conservative values, enhance control over public messaging, streamline processes for Congressional Notifications, and redirect funding towards new priorities, particularly those involving faith-based organizations and private sector partnerships."


In [106]:
doc_plot = topic_model.visualize_documents(docs)
doc_plot.show()

# Policy time line for some countries

In [82]:
country = 'Canada'
policy.group_by('year', f'Policies_by_Country/{country}.json', f'Policies_by_Country/{country}')
policies = []

years = [item for item in range(2021, 2026)]
for year in years:
  if os.path.exists(f'Policies_by_Country/{country}/{year}.json'):
    policies.extend(util.load_json(f'Policies_by_Country/{country}/{year}.json'))

vectorizer_model = CountVectorizer(
    stop_words='english',   
    ngram_range=(1,3),
    min_df=2       
)

docs = [policy["policy"]+policy["effect"] for policy in policies]
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

umap_model = umap.UMAP(
    n_neighbors=2,
    n_components=2,
    min_dist=0.0,
    metric='cosine',
    random_state=42,
    init="random" 
)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
hdbscan_model = HDBSCAN(min_cluster_size=2, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2))
ctfidf_model = ClassTfidfTransformer()

topic_model = BERTopic(
  n_gram_range = (1,2),
  embedding_model=embedding_model,    
  hdbscan_model=hdbscan_model,        
  vectorizer_model=vectorizer_model,  
  umap_model=umap_model,
  ctfidf_model=ctfidf_model,          
  calculate_probabilities=True,        
  verbose=True
)

topics, probs = topic_model.fit_transform(docs)
topic_model_info = topic_model.get_topic_info()

Grouped data has been saved in 'Policies_by_Country/Canada' directory.


2025-02-19 15:29:14,025 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-02-19 15:29:14,662 - BERTopic - Embedding - Completed ✓
2025-02-19 15:29:14,662 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-19 15:29:14,705 - BERTopic - Dimensionality - Completed ✓
2025-02-19 15:29:14,706 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-19 15:29:14,718 - BERTopic - Cluster - Completed ✓
2025-02-19 15:29:14,725 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-19 15:29:14,756 - BERTopic - Representation - Completed ✓


In [83]:
fig = topic_model.visualize_topics()
fig.show()

In [84]:
bar_chart = topic_model.visualize_barchart()
bar_chart.show()

In [80]:
representative_docs = topic_model.get_representative_docs(2)
df = pd.DataFrame(representative_docs)
pd.set_option('display.max_colwidth', None)
df

,0
0,"Promotion of investigation and research on climate change impacts and disaster risk reduction (DRR) measures, including the improvement of fishing port facilities and coastal disaster prevention forests, and adaptation measures for mountain areas.The policy aims to enhance resilience against climate change impacts, such as sea level rise and extreme weather events, through research, improved infrastructure, and disaster prevention strategies."
1,"More resilient and durable road networks; systematic designation of conservation forests; enhancement of community safety through DRR approaches; flood control through forest management; driftwood disaster risk mitigation; erosion control; enhancement of forest road facilities; accurate identification of Mountain Disaster Danger Zones (MDDZ) using technology.Improved public safety and security, reduced damage from mountain disasters, enhanced flood control, and better management of forest resources to mitigate disaster risks."
2,"Implement flood control projects, strengthen national land monitoring systems, promote the use of new technologies for disaster management, adopt measures against driftwood disasters, promote green belts in urban foothills, encourage safer land use and housing relocation, promote investigation and research on sediment and avalanche disasters, enhance preparedness for disaster waste management, and implement measures against inundation at airports.Improved disaster preparedness and response, enhanced safety against sediment disasters, better management of driftwood and disaster waste, and increased resilience of infrastructure against climate-induced disasters."
